In [2]:
class Door:
    color = 'white'
    def __init__(self, number):
        self.number = number

door = Door(1)
print(door.color, Door.color)

white white


Any Python object is automatically given a `__dict__` attribute, which contains its list of attributes.

In [3]:
door.__dict__

{'number': 1}

But, as you can see above, instance `door` doesn't have any attribute `color` in its attribute list. However, class `Door` does have a `color` attribute as seen below. So how come we can access `color` attribute of `door` instance?

In [4]:
Door.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'Door' objects>,
              '__doc__': None,
              '__init__': <function __main__.Door.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Door' objects>,
              'color': 'white'})

This is a job performed by the magic `__getattribute__()` method; in Python the dotted syntax automatically invokes this method so when we write `door.colour`, Python executes `door.__getattribute__('colour')`. That method performs the  _attribute  lookup_ action, i.e. finds the value of the attribute by looking in different places.

The standard implementation of `__getattribute__()` searches first the internal dictionary (`__dict__`) of an object, then the type of the object itself; in this case `door.__getattribute__('colour')` executes first `door.__dict__['colour']` and then, since the latter raises a `KeyError` exception, `door.__class__.__dict__['colour']`

In [5]:
door.color = 'blue'

Door.color

'white'

In [6]:
door.color

'blue'

In [7]:
door.__dict__

{'color': 'blue', 'number': 1}

In [8]:
door.__dict__['color'] = 'grey'
door.color

'grey'

In [9]:
Door.color

'white'

In [10]:
door.__getattribute__('color')

'grey'

When we try to assign a value to a class attribute directly on an instance, we just put in the `__dict__` of the instance a value with that name, and this value masks the class attribute since it is found first by `__getattribute__()`. As you can see from the examples of the previous section, this is different from changing the value of the attribute on the class itself.


In [11]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status
        
    def open(self):
        self.status = 'open'

In [12]:
Door.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'Door' objects>,
              '__doc__': None,
              '__init__': <function __main__.Door.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Door' objects>,
              'colour': 'brown',
              'open': <function __main__.Door.open>})

In [13]:
door = Door(1,'closed')
door.__dict__

{'number': 1, 'status': 'closed'}

Just like class attributes, methods are listed in class `__dict__` but not in instance `__dict__`, so we might guess that they can be accessed like attributes.

In [14]:
Door.open is door.open

False

Well, that wasn't the case. Let's investigate further -

In [15]:
Door.open

<function __main__.Door.open>

In [16]:
door.open

<bound method Door.open of <__main__.Door object at 0x03666790>>

So, the class method is listed in the members dictionary as function. So far, so good. Taking it from the instance returns a bound method.

A function is a procedure you named and defined with the `def` statement. When you refer to a function as part of a class in Python 3 you get a plain function, without any difference from a function defined outside a class.

When you get the function from an instance, however, it becomes a bound method. The name method simply means "a function inside an object", according to the usual OOP definitions, while bound signals that the method is linked to that instance. Why does Python bother with methods being bound or not? And how does Python transform a function into a bound method?

First of all, if you try to call a class function you get an error:

In [17]:
try:
    Door.open()
except TypeError as e:
    print(e)

open() missing 1 required positional argument: 'self'


Yes. Indeed the function was defined to require an argument called 'self', and calling it without an argument raises an exception. This perhaps means that we can give it one instance of the class and make it work


In [18]:
Door.open(door)

In [19]:
door.status

'open'

Python does not complain here, and the method works as expected. So `Door.open(door)` is the same as `door.open()`, and this is the difference between a plain function coming from a class an a bound method: the bound method automatically passes the instance as an argument to the function.

Again, under the hood, `__getattribute__()` is working to make everything work and when we call `door.open()`, Python actually calls `door.__class__.open(door)`. However, `door.__class__.open` is a plain function, so there is something more that converts it into a bound method that Python can safely call.

When you access a member of an object, Python calls `__getattribute__()` to satisfy the request. This magic method, however, conforms to a procedure known as descriptor protocol. For the read access `__getattribute__()` checks if the object has a `__get__()` method and calls this latter. So the converstion of a function into a bound method happens through such a mechanism. Let us review it by means of an example.


In [20]:
dir(door.__class__.__dict__['open'])

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [22]:
door.__class__.__dict__['open'].__get__

<method-wrapper '__get__' of function object at 0x036566A8>

In [24]:
door.__class__.__dict__['open'].__get__(door)

<bound method Door.open of <__main__.Door object at 0x03666790>>

and we get exactly what we were looking for. This complex syntax is what happens behind the scenes when we call a method of an instance.

In [26]:
type(door.open), type(Door.open)

(method, function)

As you can see, Python tells the two apart recognizing the first as a function and the second as a method, where the second is a function bound to an instance.

What if we want to define a function that operates on the class instead of operating on the instance? As we may define class attributes, we may also define class methods in Python, through the *classmethod decorator*. Class methods are functions that are bound to the class and not to an instance.

In [27]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status

    @classmethod
    def knock(cls):
        print("Knock!")

    def open(self):
        self.status = 'open'
        
    def close(self):
        self.status = 'closed'

Such a definition makes the method callable on both the instance and the class

In [28]:
door = Door(1, 'closed')
door.knock()

Knock!


In [29]:
Door.knock()

Knock!


In [30]:
type(door.knock), type(Door.knock)

(method, method)

Python identifies both as bound methods -

In [31]:
door.knock, Door.knock

(<bound method Door.knock of <class '__main__.Door'>>,
 <bound method Door.knock of <class '__main__.Door'>>)

In [32]:
door.__class__.__dict__['knock']

As you can see the `knock()` function accepts one argument, which is called `cls` just to remember that it is not an instance but the class itself. This means that inside the function we can operate on the class, and the class is shared among instances.

In [33]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status

    @classmethod
    def knock(cls):
        print("Knock!")

    @classmethod
    def paint(cls, colour):
        cls.colour = colour

    def open(self):
        self.status = 'open'
        
    def close(self):
        self.status = 'closed'

In [34]:
door = Door(1, 'closed')

In [35]:
Door.colour, door.colour

('brown', 'brown')

In [36]:
Door.paint('white')           #class method applied on class
Door.colour, door.colour      #both class and instance will be affected

('white', 'white')

In [38]:
door.paint('blue')            #class method applied to instance
Door.colour, door.colour      #both class and instance will be affected

('blue', 'blue')

The class method can be called on the class, but this affects both the class and the instances, since the colour attribute of instances is taken at runtime from the shared class.

Class methods can be called on instances too, however, and their effect is the same as before. The class method is bound to the class, so it works on this latter regardless of the actual object that calls it (class or instance).

#### Delegation, Composition and Inheritance

In [39]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status

    @classmethod
    def knock(cls):
        print("Knock!")

    @classmethod
    def paint(cls, colour):
        cls.colour = colour

    def open(self):
        self.status = 'open'
        
    def close(self):
        self.status = 'closed'
        
class SecurityDoor(Door):
    pass

In [40]:
sdoor = SecurityDoor(1,'closed')

In [41]:
SecurityDoor.colour is Door.colour

True

In [42]:
sdoor.colour is Door.colour

True

In [43]:
sdoor.__dict__

{'number': 1, 'status': 'closed'}

In [44]:
sdoor.__class__.__dict__

mappingproxy({'__doc__': None, '__module__': '__main__'})

In [45]:
Door.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'Door' objects>,
              '__doc__': None,
              '__init__': <function __main__.Door.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Door' objects>,
              'close': <function __main__.Door.close>,
              'colour': 'brown',
              'knock': <classmethod at 0x3666510>,
              'open': <function __main__.Door.open>,
              'paint': <classmethod at 0x3666b30>})

As you can see the content of `__dict__` for `SecurityDoor` is very narrow compared to that of `Door`. The inheritance mechanism takes care of the missing elements by climbing up the classes tree. Where does Python get the parent classes? **A class always contains a `__bases__` tuple that lists them**

In [46]:
SecurityDoor.__bases__

(__main__.Door,)

So an example of what Python does to resolve a class method call through the inheritance tree is

In [47]:
sdoor.__class__.__bases__[0].__dict__['knock'].__get__(sdoor)

<bound method Door.knock of <class '__main__.SecurityDoor'>>

In [48]:
sdoor.knock

<bound method Door.knock of <class '__main__.SecurityDoor'>>

Let us try now to override some methods and attributes. In Python you can override (redefine) a parent class member simply by redefining it in the child class.

In [49]:
class SecurityDoor(Door):
    colour = 'gray'
    locked = True
    
    def open(self):
        if not self.locked:
            self.status = 'open'

In [50]:
SecurityDoor.__dict__

mappingproxy({'__doc__': None,
              '__module__': '__main__',
              'colour': 'gray',
              'locked': True,
              'open': <function __main__.SecurityDoor.open>})

So when you override a member, the one you put in the child class is used instead of the one in the parent class simply because the former is found before the latter while climbing the class hierarchy. This also shows you that Python does not implicitly call the parent implementation when you override a method. So, overriding is a way to block implicit delegation.

If we want to call the parent implementation we have to do it explicitly. In the former example we could write


In [51]:
class SecurityDoor(Door):
    colour = 'gray'
    locked = True
    
    def open(self):
        if self.locked:
            return
        Door.open(self)

In [52]:
sdoor = SecurityDoor(1,'closed')
sdoor.status

'closed'

In [53]:
sdoor.open()
sdoor.status

'closed'

In [54]:
sdoor.locked = False
sdoor.open()
sdoor.status

'open'

This form of explicit parent delegation is heavily discouraged, however.

The first reason is because of the very high coupling that results from explicitly naming the parent class again when calling the method. Coupling, in the computer science lingo, means to link two parts of a system, so that changes in one of them directly affect the other one, and is usually avoided as much as possible. In this case if you decide to use a new parent class you have to manually propagate the change to every method that calls it. Moreover, since in Python the class hierarchy can be dynamically changed (i.e. at runtime), this form of explicit delegation could be not only annoying but also wrong.

The second reason is that in general you need to deal with multiple inheritance, where you do not know a priori which parent class implements the original form of the method you are overriding.

To solve these issues, Python supplies the `super()` built-in function, that climbs the class hierarchy and returns the correct class that shall be called. The syntax for calling `super()` is


In [55]:
class SecurityDoor(Door):
    colour = 'gray'
    locked = True
    
    def open(self):
        if self.locked:
            return
        super().open()

The output of `super()` is not exactly the `Door` class. It returns a `super`. More on `super()` has been discussed in a separate notebook.

##### Composition

Composition means that an object knows another object, and explicitly delegates some tasks to it. While inheritance is implicit, composition is explicit.

First of all let us implement classic composition, which simply makes an object part of the other as an attribute

In [56]:
class SecurityDoor:
    colour = 'gray'
    locked = True
    
    def __init__(self, number, status):
        self.door = Door(number, status)  #take note of this
        
    def open(self):
        if self.locked:
            return
        self.door.open()
        
    def close(self):
        self.door.close()

The primary goal of composition is to relax the coupling between objects. This little example shows that now `SecurityDoor` is an `object` and no more a `Door`, which means that the internal structure of `Door` is not copied. For this very simple example both `Door` and `SecurityDoor` are not big classes, but in a real system objects can very complex; this means that their allocation consumes a lot of memory and if a system contains thousands or millions of objects that could be an issue.

The composed `SecurityDoor` has to redefine the `colour` attribute since the concept of delegation applies only to methods and not to attributes, doesn't it?

Well, no. Python provides a very high degree of indirection for objects manipulation and attribute access is one of the most useful. As you already discovered, accessing attributes is ruled by a special method called `__getattribute__()` that is called whenever an attribute of the object is accessed. Overriding `__getattribute__()`, however, is overkill; it is a very complex method, and, being called on every attribute access, any change makes the whole thing slower.

The method we have to leverage to delegate attribute access is `__getattr__()`, which is a special method that is called whenever the requested attribute is not found in the object. So basically it is the right place to dispatch all attribute and method access our object cannot handle. The previous example becomes

In [57]:
class SecurityDoor:
    locked = True
    
    def __init__(self, number, status):
        self.door = Door(number, status)
        
    def open(self):
        if self.locked:
            return
        self.door.open()
        
    def __getattr__(self, attr):
        return getattr(self.door, attr)

Using `__getattr__()` blends the separation line between inheritance and composition since after all the former is a form of automatic delegation of every member access.

In [58]:
class ComposedDoor:
    def __init__(self, number, status):
        self.door = Door(number, status)
        
    def __getattr__(self, attr):
        return getattr(self.door, attr)

As this last example shows, delegating every member access through `__getattr__()` is very simple. Pay attention to `getattr()` which is different from `__getattr__()`. The former is a built-in that is equivalent to the dotted syntax, i.e. `getattr(obj, 'someattr')` is the same as `obj.someattr`, but you have to use it since the name of the attribute is contained in a string.

Composition provides a superior way to manage delegation since it can selectively delegate the access, even mask some attributes or methods, while inheritance cannot. In Python you also avoid the memory problems that might arise when you put many objects inside another; Python handles everything through its reference, i.e. through a pointer to the memory position of the thing, so the size of an attribute is constant and very limited.

#### Polymorphism

In [59]:
class Room:
    def __init__(self, door):
        self.door = door
        
    def open(self):
        self.door.open() #this part was tricky for me. Here `door` is instance of another class

    def close(self):
        self.door.close()

    def is_open(self):
        return self.door.is_open()

A very simple class, as you can see, just enough to exemplify polymorphism. The `Room` class accepts a `door` variable, and the type of this variable is not specified. Duck typing in action: the actual type of `door` is not declared, there is no "acceptance test" built in the language. Indeed, the incoming variable shall export the following methods that are used in the `Room` class: `open(), close(), is_open()`. So we can build the following classes


In [61]:
class Door:
    def __init__(self):
        self.status = "closed"

    def open(self):
        self.status = "open"

    def close(self):
        self.status = "closed"

    def is_open(self):
        return self.status == "open"


class BooleanDoor:
    def __init__(self):
        self.status = True

    def open(self):
        self.status = True

    def close(self):
        self.status = False

    def is_open(self):
        return self.status

Both represent a door that can be open or closed, and they implement the concept in two different ways: the first class relies on strings, while the second leverages booleans. Despite _being_ two different types, both _act_ the same way, so both can be used to build a `Room` object.

In [62]:
door = Door()
bool_door = BooleanDoor()
room = Room(door)
bool_room = Room(bool_door)

In [63]:
room.open()
print(room.is_open())

True


In [64]:
room.close()
print(room.is_open())

False


In [65]:
bool_room.open()
print(bool_room.is_open())

True


In [66]:
bool_room.close()
print(bool_room.is_open())

False


The following code is helpful in understanding the above examples under Polymorphism. 

In [73]:
class Foo:
    def __init__(self, door):
        self.door = door
        
    def test(self):
        return self.door.bar()
    
class Baz:
    def bar(self):
        print('Hi')
        
        
baz = Baz()
foo = Foo(baz)
foo.test()        

Hi


In [4]:
class Test:
    def __init__(self):
        print(self)
        return 1
    
test = Test()    

TypeError: __init__() should return None, not 'int'

In [5]:
test

In [6]:
class Test:
    def __init__(self):
        print(self)
    
test = Test()    


In [7]:
test

In [2]:
class Test:
    def __init__(self, name):
        self.name = name
        
    def setmethod(self, value):
        self.value = value
        
test =  Test("Sam")
Test.setmethod(test, 3)

print(type(test.setmethod))
print(type(Test.setmethod))

print(Test.setmethod(test, 3) == test.setmethod(3))

<class 'method'>
<class 'function'>
True


In [3]:
class Person:
    def __init__(self, name):
        self.name = name
        print(self.name)
        
    def sayhello(self):
        print("Hi", self.name)
        
    def __del__(self):
        print("bye", self.name)
        
A = Person("Mayank")

A.sayhello()        
        

Mayank
Hi Mayank


In [4]:
del A


bye Mayank


In [5]:
class A:
    i = 123
    def __init__(self):
        self.i = 1234
        
    def f(self):
        print("Hi")
        
print(A.i)
print(A().i)
print(A().f());

123
1234
Hi
None


As is clear from above example, `__init__` function always return `None`

In [6]:
class Test:
    def __init__(self):
        print(self)
        print('x')

test = Test()        

x


`__init__` function is executed as soon as an instance of a class created.

In [7]:
class Test:
    def __init__(self):
        print("Test id is %d" %id(self)) 
        
x = Test()
y = Test()*10

Test id is 52114128
Test id is 52055504


TypeError: unsupported operand type(s) for *: 'Test' and 'int'

This program shows that object is instantiated before it is assigned to any variable. Line 1 runs fine. Line 2 produces `TypeError` because of wrong multiplication. Here instance was created (as shown by `id()` function) but its product with 10 produced error and nothing was assigned to `y`. Error happens only after instance is created and before instance is assigned a name 'y'. If this program is run interactively, `print(x)` command will run successfully but `print(y)` command will raise `NameError`. 

In [8]:
print(x)
print(y)

NameError: name 'y' is not defined

In [9]:
class Strings:
    def __init__(self,a):
        self.a = a
        
    def __add__(self, other):
        return self.a + other.a
    
a = Strings('x')
b = Strings('y')

a+b

'xy'

In [10]:
class Strings:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __add__(self, other):
        self.x = self.x + other.x
        self.y = self.y + other.y
        return self.x, self.y
    
a = Strings(3,4)
b = Strings(4,5)

a + b


(7, 9)

In [11]:
class Employee:
    'common base class for all employees'
    empCount = 0
    
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary
        Employee.empCount += 1
        
    def displayCount(self):
        print("Total No of Employee: %d" %Employee.empCount)
        
    def displayEmployee(self):
        print("Name: ", self.name, "Salary: ", self.salary)
        
emp1 = Employee('Sam',200)
emp2 = Employee('Peter',300)

print(Employee.empCount)
emp1.displayCount()
emp2.displayCount()
emp1.displayEmployee()
emp2.displayEmployee()


2
Total No of Employee: 2
Total No of Employee: 2
Name:  Sam Salary:  200
Name:  Peter Salary:  300


In [12]:
class C:
    def setname(self, who):
        self.name = who
        
I = C()
I.setname('Mayank')
I.name

'Mayank'

In [13]:
class C:
    def __init__(self,who):
        self.name = who
        
I = C('Mayank')    
I.name

'Mayank'

Here `__init__` is used instead of `setname`. This method is automatically called, whenever an instance is created. No need for extra method calls.

In [14]:
class Test:
    def __init__(self,arg1,arg2):
            self.arg1 = arg1
            self.arg2 = arg2
    
    def printargs(self):
        print(self.arg1)
        print(self.arg2)
        
instance = Test('a','b')

instance.printargs()
    

a
b


In [15]:
Test.printargs(instance)

a
b


Notice that `instance.printargs()` is equivalent to `Test.printargs(instance)`

In [16]:
class Employee:
    
    empCount = 0
    
    def __init__(self):
        Employee.empCount += 1
        print(Employee.empCount)
        print(id(self))

    def __del__(self):
        Employee.empCount -= 1
        print(Employee.empCount)
        print(id(self))
        
emp1 = Employee()
emp2 = Employee()

1
54026032
2
52114064


In [17]:
del emp1

1
54026032


In [18]:
del emp2

0
52114064


In [19]:
Employee.empCount

0

#### Encapsulation, Private Attributes and Data Hiding

In [20]:
class Encapsulation(object):
    def __init__(self, a, b, c):
        self.public = a
        self._protected = b
        self.__private = c
        print(self.__private)
        
A = Encapsulation(1,2,3)        

3


In [21]:
print(A.public)
print(A._protected)
print(A.__private)

1
2


AttributeError: 'Encapsulation' object has no attribute '__private'

As is clear from above, private attributes can't be accessed from outside. It can only be accessed from inside of class.

In [22]:
class Foo:
    def __init__(self, name):
        self.__name = name
    def setname(self):
        return self.__name
    
bar = Foo('Mayank')

bar.setname()

'Mayank'

In [23]:
bar.__name

AttributeError: 'Foo' object has no attribute '__name'

In [24]:
class Foo:
    def __init__(self, name):
        self.name = name
    def setname(self):
        return self.name
    
bar = Foo('Mayank')

bar.setname()

'Mayank'

In [25]:
bar.name

'Mayank'

Compare above 2 example. In first example, `bar.__name` raised `AttributeError` while in second example, `bar.name` ran correctly. 

In [26]:
class D:
    value =  5
    
class E(D):
    def printf(self):
        print(self.value)
        
a = E()
a.printf()

5


In [27]:
class D:
    __value =  5
    
class E(D):
    def printf(self):
        print(self.__value)
        
a = E()
a.printf()

AttributeError: 'E' object has no attribute '_E__value'

This illustrates that private attributes can't be accessed by child class as well.

From Python Essential Reference book – 

By default, all attributes and methods of a class are “public.”This means that they are all accessible without any restrictions. It also implies that everything defined in a base class is inherited and accessible within a derived class.This behavior is often undesirable in object-oriented applications because it exposes the internal implementation of an object and can lead to namespace conflicts between objects defined in a derived class and those defined in a base class. To fix this problem, all names in a class that start with a double underscore, such as` __Foo`, are automatically mangled to form a new name of the form `_Classname__Foo`. This effectively provides a way for a class to have private attributes and methods because private names used in a derived class won’t collide with the same private names used in a base class. 

Here is an example -
```python
class A(object):
    def __init__(self):
        self.__X = 3       # Mangled to self._A__X
    def __spam(self):      # Mangled to _A__spam()
        pass
    def bar(self):
        self.__spam()      # Only calls A.__spam()

class B(A):
    def __init__(self):
        A.__init__(self)
        self.__X = 37      # Mangled to self._B__X
    def __spam(self):      # Mangled to _B__spam()
        pass
```

Although this scheme provides the illusion of data hiding, there’s no strict mechanism in place to actually prevent access to the “private” attributes of a class. In particular, if the name of the class and corresponding private attribute are known, they can be accessed using the mangled name.


In [28]:
class Employee:
    def __init__(self, name):
        self.name = name
        
    def grade(self):
        print("Diff grade for diff group")
        
        
class Top(Employee):
    def __init__(self, name, perk):
        #Employee.__init__(self, name) #code is running even without this line. Why?
        self.perk = perk
    
    def grade(self):    
            Employee.grade(self)
            print("Top grade")
        
class Middle(Employee):
    def __init__(self, name, salary):
        Employee.__init__(self, name)
        self.salary =  salary
        
    def grade(self):
        print("Middle Grade")
        

emp1 = Top('Sam', 100)
emp2 = Middle('Peter', 200)

for e in [emp1, emp2]:
    print(e.grade());
    

Diff grade for diff group
Top grade
None
Middle Grade
None


In [30]:
class Customer:
    def __init__(self, name, balance):
        self.name = name
        self.balance =balance
    
    def deposit(self, amount):
        self.amount = amount
        self.balance = self.balance + amount
        print(self.balance)
        
    def test(self):
        print(amount)
        print(self.amount)
        
        
customer1 = Customer("Mayank", 1000)
customer2 = Customer("Kajal", 500)

customer1.deposit(500)
customer2.deposit(500)
    
    

1500
1000


In [31]:
customer1.test()

NameError: name 'amount' is not defined

'amount' can only be accessed by 'deposit' method. Here self.balance is called instance variable and amount would be local variable. Instance variable can be accessed by all methods defined in a class while local variables can be used by only that method in which they are defined.

In [32]:
class Test():
    pass

class Student(Test):
    pass

print(Test.__mro__)
print(Student.__mro__)

(<class '__main__.Test'>, <class 'object'>)
(<class '__main__.Student'>, <class '__main__.Test'>, <class 'object'>)


In [33]:
issubclass(Test, object)

True

In [34]:
issubclass(Student, object)

True

In [35]:
def foo():
    pass

issubclass(foo.__class__, object)

True

#### Static Method

In [36]:
class Test:

    @staticmethod
    def testmethod():
        print('Hi')
    
    def __init__(self,number):
        self.number = number

    def setmethod(self):
        print(self.number)    

Test.testmethod()       

Hi


#### Class Method

In [64]:
class Times():
    factor = 1
    @classmethod
    def mul(cls,x):
        return cls.factor*x

class TwoTimes(Times):
    factor = 2

x = TwoTimes.mul(4)    

print(x)

8


#### Properties

In [66]:
import math

class Circle:
    def __init__(self,radius):
        self.radius = radius

    @property
    def area(self):
        return math.pi*self.radius**2

    @property
    def perimeter(self):
        return 2*math.pi*self.radius

c = Circle(4)
c.radius



4

In [67]:
c.radius = 5
c.radius

5

In [68]:
c.perimeter

31.41592653589793

In [69]:
c.area

78.53981633974483

In [70]:
c.perimeter =  5

AttributeError: can't set attribute

As we know, instances of a class has attributes and methods bound to them. To access attributes and methods, we use dot notation. To access an attribute, we write `object.attribute` while to apply a method, we write `object.method()`. However, unlike methods, we can always change the value of attribute as shown above (c.radius = 5). In above program, if we don’t use `@property` decorator, we would apply method like this: `c.perimeter()`. But by using `@property` decorator, we get rid of those parentheses used after a method. So, now we can write `c.perimeter` instead of `c.perimeter()`. However, as said earlier, we can’t change the value of `c.perimeter` which is obvious. This is an example of making programming interface as uniform as possible.(Uniform Access Principle)	

#### `__repr__`

In [72]:
class Test:
	def __init__(self,value):
		self.value = value
	def __repr__(self):
		return 'value for current instance= %d' % self.value

a = Test(5)
a

value for current instance= 5

#### `__call__`

In [73]:
class Foo:
    def __call__(self):
        print('Hi')
        
bar = Foo()
bar()


Hi
